# Updating Filevine Contact Geocodes

In [2]:
import numpy as np
import pandas as pd
import json
import httpx

In [3]:
# Load API credentials from secrets.json
with open("../secrets.json", "r") as file:
    secrets = json.load(file)

# Extract credentials
client_id = secrets["client_id"]
client_secret = secrets["client_secret"]
url = secrets["token_url"]
pat = secrets["pat_token"]

# Set up the data payload
data = {
    "grant_type": "personal_access_token",
    "token": pat,
    "client_id": client_id,
    "client_secret": client_secret,
    "scope": "fv.api.gateway.access tenant filevine.v2.api.* openid email fv.auth.tenant.read"
}
    
# Make the POST request
response = httpx.post(url, data=data, headers={"Content-Type": "application/x-www-form-urlencoded"})

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    token_info = response.json()
    access_token = token_info.get("access_token")
    # print("Access Token:", access_token)
    print("✅ Access Token obtained successfully")
else:
    print("❌ Failed to obtain access token:", response.text)  

# Define the API endpoint
USER_ORG_ENDPOINT = "https://api.filevineapp.com/fv-app/v2/utils/GetUserOrgsWithToken"

# Set up headers with the access token
headers = {
    "Authorization": f"Bearer {access_token}"
}

# Make the POST request
response = httpx.post(USER_ORG_ENDPOINT, headers=headers)

# Check the response
if response.status_code == 200:
    user_info = response.json()
    user_id = user_info['user']['userId']['native']
    org_id = user_info['orgs'][0]['orgId']
    print(f"✅ User ID: {user_id}")
    print(f"✅ Org ID: {org_id}")
else:
    print(f"❌ Failed to retrieve User ID and Org ID: {response.text}")

# -----------------------------
# Configuration & API Endpoints
# -----------------------------

# Global headers including GZIP compression support.
headers = {
    "Authorization": f"Bearer {access_token}",
    "x-fv-userid": str(user_id),
    "x-fv-orgid": str(org_id),
    "Content-Type": "application/json",
    "Accept-Encoding": "gzip, deflate",
    "Accept": "application/json"
}

✅ Access Token obtained successfully
✅ User ID: 990018081
✅ Org ID: 990000379


In [ ]:
df = pd.read_excel('./data/outbound_geolocated_reviewed.xlsx',
                    usecols = ['contactId', 'Full Name', 'Full Address','Latitude','Longitude'])
df

,contactId,Full Name,Full Address,Latitude,Longitude
0,996701857,Waldorf Total Health Chiropractic & Physical T...,"12102 Old Line Center, Waldorf, MD 20602",38.616638,-76.890698
1,996605635,Bezak Chiropractic And Rehabilitation,"7500 Hanover Parkway, Greenbelt, MD 20770",38.992915,-76.875826
2,996356411,Dunkirk Chiropractic & Wellness Center,"10020 Southern Maryland Blvd, Dunkirk, MD 20754",38.714689,-76.659379
3,996605523,"Effective Integrative Healthcare - Lanham, Md","7400 Riverdale Rd, Lanham, MD 20706",38.962083,-76.884654
4,996616309,Rebound Integrative Health & Sports Clinic,"7701 Greenbelt Rd, Greenbelt, MD 20770-6521",38.994033,-76.874777
...,...,...,...,...,...
58,996701927,"Maryland Healthcare Clinics - Owings Mills, Md","5 Park Center Court, Owings Mills, MD 21117",39.399382,-76.753732
59,996465882,Washington Wellness,"25 Massachusetts Ave Nw, Washington, DC 20001-...",38.898473,-77.011358
60,996593018,Neuropro Concussion Clinic,"1110 Benfield Blvd, Millersville, MD 21108",39.094464,-76.631838
61,996668361,Rebound Integrative Health & Sports Clinic - T...,"3611 Branch Avenue, Temple Hills, MD 20748",38.844634,-76.950739
